# Obsidian to LaTeX Converter - Jupyter Interface

This notebook provides an interactive interface to convert Obsidian markdown files to LaTeX sections.

In [ ]:
# Import necessary libraries
import os
import sys
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Add the parent directory to the path to import the obsidian_to_latex package
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the converter
from obsidian_to_latex.converter import ObsidianLatexSectionConverter

## File Selection Widgets

Select your Obsidian markdown file and output options.

In [ ]:
# Get the current directory
current_dir = os.getcwd()

# Function to list available markdown files
def list_markdown_files(directory=current_dir):
    md_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.md'):
                md_files.append(os.path.join(root, file))
    return md_files

# Create widgets
input_file_dropdown = widgets.Dropdown(
    options=list_markdown_files(),
    description='Input File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

output_file = widgets.Text(
    value='output.tex',
    description='Output File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

figures_dir = widgets.Text(
    value='figures',
    description='Figures Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

level_adjust = widgets.IntSlider(
    value=0,
    min=-2,
    max=2,
    step=1,
    description='Header Level Adjustment:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

verbose_checkbox = widgets.Checkbox(
    value=False,
    description='Verbose Mode',
    style={'description_width': 'initial'}
)

# Display widgets
display(input_file_dropdown, output_file, figures_dir, level_adjust, verbose_checkbox)

## Convert Button and Output

In [ ]:
# Create output widgets
output_area = widgets.Output()
preview_area = widgets.Output()

# Create conversion function
def convert_file(b):
    # Clear previous output
    with output_area:
        clear_output()
    with preview_area:
        clear_output()
    
    # Get values from widgets
    input_file = input_file_dropdown.value
    output_file_path = output_file.value
    figures_directory = figures_dir.value
    level_adjustment = level_adjust.value
    verbose = verbose_checkbox.value
    
    # Create converter
    converter = ObsidianLatexSectionConverter(
        input_file=input_file,
        output_file=output_file_path,
        figures_dir=figures_directory,
        verbose=verbose
    )
    
    # Perform conversion
    with output_area:
        try:
            print(f"Converting {input_file} to {output_file_path}...")
            latex_content = converter.convert(level_adjustment=level_adjustment)
            
            if latex_content:
                success = converter.save(latex_content)
                if success:
                    converter.process_images()
                    print(f"Conversion completed successfully!")
                    print(f"Output saved to: {os.path.abspath(output_file_path)}")
                    
                    # Show LaTeX Preview
                    with preview_area:
                        print("LaTeX Preview (first 500 characters):")
                        print("\n" + "=" * 80 + "\n")
                        preview_content = latex_content[:500]
                        if len(latex_content) > 500:
                            preview_content += "\n... (content truncated)"
                        print(preview_content)
                else:
                    print("Failed to save output file.")
            else:
                print("Conversion failed.")
        except Exception as e:
            print(f"An error occurred: {e}")

# Create convert button
convert_button = widgets.Button(
    description='Convert to LaTeX',
    button_style='primary',
    tooltip='Convert Obsidian markdown to LaTeX'
)

convert_button.on_click(convert_file)

# Display button and output areas
display(convert_button)
display(widgets.HTML("<h3>Log Output:</h3>"))
display(output_area)
display(widgets.HTML("<h3>LaTeX Preview:</h3>"))
display(preview_area)

## Tips for Using the Converter

1. **Header Levels**: Use the header level adjustment to fit your LaTeX document structure. For example, if your Obsidian file uses `#` headers that should be `\subsection` in LaTeX, set the adjustment to `1`.

2. **Images**: The converter will try to find images in the following locations:
   - `attachments` folder in the same directory as the markdown file
   - `assets` folder in the same directory
   - `images` folder in the same directory
   - `attachments` folder in the parent directory

3. **Math Expressions**: The converter preserves math expressions in the document. It handles both inline math (`$...$`) and display math (`$$...$$`).

4. **Including in LaTeX Documents**: To include the generated section in your LaTeX document, use `\input{path/to/output.tex}` in your main LaTeX file.